# i) Regra da soma

In [25]:
import os
import numpy as np
import pandas as pd
import clustering

## Datasets

In [26]:
DATA_BASE_PATH = "./data"

FAC_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fac")
FOU_FILE = os.path.join(DATA_BASE_PATH, "mfeat-fou")
KAR_FILE = os.path.join(DATA_BASE_PATH, "mfeat-kar")

fac = np.loadtxt(FAC_FILE, dtype=int)
fou = np.loadtxt(FOU_FILE, dtype=float)
kar = np.loadtxt(KAR_FILE, dtype=float)

## Importando melhor resultado

In [27]:
best_result = clustering.import_best_result("data/melhor_resultado_todas.pickle")
partition, y_true = clustering.get_hard_patitions(best_result["membership_degree"])


## Probabilidade à priori das classes

In [28]:
Pw = np.array([len(c)/2000 for c in partition])
Pw

array([0.1175, 0.116 , 0.1355, 0.1465, 0.0405, 0.01  , 0.1305, 0.151 ,
       0.0815, 0.071 ])

## Função de densidade

In [29]:
# def calc_gaussian_density_prob(xk, d, means, var, cov_matrix):
#     coef = np.power(2*np.pi, -d/2) 
#     inv_cov_matrix = np.linalg.inv(cov_matrix)
#     (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
#     sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
#     diff = xk - means
#     exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
#     exp_func = np.exp(exp_exp)
    
#     return coef * sqrt_det_inv_cov * exp_func

In [30]:

# def calc_gaussian_bayesian_data(x, partition):
#     n,d = x.shape
#     qtd_w = len(partition)
#     means = np.array([x[idxs].mean(axis=0) for idxs in partition])
#     var = np.array([((x[idxs]-means[i])**2).mean(axis=0) for i, idxs in enumerate(partition)])
#     cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

#     for i in range(qtd_w):
#         np.fill_diagonal(cov_matrix[i], var[i])
                
#     p_x_w = np.empty((n, qtd_w))
    
#     for k in range(n):
#         for i in range(qtd_w):
#             p_x_w[k, i] = calc_gaussian_density_prob(x[k], d, means[i], var[i], cov_matrix[i])   
    
#     return p_x_w

def calc_prob_posteriori(p_x_w, Pw):
    qtd_x, qtd_w = p_x_w.shape 
    p_w_x = np.empty((qtd_x, qtd_w))
    
    for k in range(qtd_x):
        for i in range(qtd_w):
            sum_all = np.dot(p_x_w[k], Pw)
            p_w_x[k,i] = (p_x_w[k,i] * Pw[i])/sum_all
       
    return p_w_x


## Densidades por dataset

In [31]:
# fac_byn_density_probs = calc_gaussian_bayesian_data(fac, partition)
# fou_byn_density_probs = calc_gaussian_bayesian_data(fou, partition)
# kar_byn_density_probs = calc_gaussian_bayesian_data(kar, partition)

In [32]:
# fac_byn_density_probs[0,0], fou_byn_density_probs[0,0], kar_byn_density_probs[0,0]

## Prob. à posteriori por view

In [33]:
# fac_byn_posteriori_probs = calc_prob_posteriori(fac_byn_density_probs, Pw)
# fou_byn_posteriori_probs = calc_prob_posteriori(fou_byn_density_probs, Pw)
# kar_byn_posteriori_probs = calc_prob_posteriori(kar_byn_density_probs, Pw)


## Regra da soma

Precisei tirar o fac porque seus valores são nulos. TENTAR CORRIGIR ISSO

In [34]:
def regra_soma_padrao(views, Pw):
    qtd_x = views[0].shape[0]
    qtd_w = len(Pw)
    x_sum_w = np.empty((qtd_x, qtd_w))

    for i in range(qtd_x):
        for k in range(qtd_w):
            views_sum = sum([v[i,k] for v in views])
            x_sum_w[i,k] = (1-len(views))*Pw[k] + views_sum
                    
    y_pred = x_sum_w.argmax(axis = 1) 
    return y_pred

## Estimador do scikit

In [35]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class ClassificaforBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
        self.partition = partition
        self.Pw = Pw
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self._fit_gaussian_bayesian_data(X)
        self.X_ = X
        self.y_ = y
        return self
    
    def _fit_gaussian_bayesian_data(self, X):
        n, d = X.shape
        qtd_w = len(self.partition)
        self.means = np.array([X[idxs].mean(axis=0) for idxs in self.partition])
        self.var = np.array([((X[idxs]-self.means[i])**2).mean(axis=0) for i, idxs in enumerate(self.partition)])
        self.cov_matrix = [np.zeros((d,d)) for _ in range(qtd_w)]

        for i in range(qtd_w):
            np.fill_diagonal(self.cov_matrix[i], self.var[i])

        return self
    
    def _calc_gaussian_density_prob(self, xk, cls):
        d = xk.shape[0]
        coef = np.power(2*np.pi, -d/2)
        inv_cov_matrix = np.linalg.inv(self.cov_matrix[cls])
        (sign, logdet) = np.linalg.slogdet(inv_cov_matrix)
        sqrt_det_inv_cov = np.sqrt(sign*np.exp(logdet))
        diff = xk - self.means[cls]
        exp_exp = np.dot((-1/2)*np.dot(diff.T, inv_cov_matrix), diff)
        exp_func = np.exp(exp_exp)

        return coef * sqrt_det_inv_cov * exp_func

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        
        desity_probs = np.empty((X.shape[0], len(self.classes_)))
        for k in range(desity_probs.shape[0]):
            for j in range(len(self.classes_)):
                desity_probs[k,j] = self._calc_gaussian_density_prob(X[k], j)
        
        post_probs = calc_prob_posteriori(desity_probs, self.Pw)
        
        return post_probs
        
        

In [36]:
class RegraSomaClasificadorBayesiano(BaseEstimator, ClassifierMixin):
    def __init__(self, partition, Pw):
#         self.views = views
        self.partition = partition
        self.Pw = Pw
        self.clfs = []        
    
    def fit(self, X, y):
#         X, y = check_X_y(X, y)
         
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y
        
        for x in X:
            clf = ClassificaforBayesiano(self.partition, Pw)
            clf.fit(x,y)
            self.clfs.append(clf)
            
        return self
    
    def predict(self, X):
        assert len(X) == len(self.clfs)
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
#         X = check_array(X)
        
        post_probs = [clf.predict_proba(x) for clf, x in zip(self.clfs, X)]
        
        return self.regra_soma(post_probs, Pw=self.Pw)
    
    def get_params(self, deep=True):
        return {"Pw": self.Pw, "partition": self.partition}
    
    def regra_soma(self, matrizes, Pw):
        qtd_x = matrizes[0].shape[0]
        qtd_w = len(Pw)
        x_sum_w = np.empty((qtd_x, qtd_w))
        
        for i in range(qtd_x):
            for k in range(qtd_w):
                views_sum = sum([v[i,k] for v in matrizes])
                # views_sum =  fac_p_w_x[i,k] fou_p_w_x[i,k] + kar_p_w_x[i,k]
                x_sum_w[i,k] = (1-len(matrizes))*Pw[k] + views_sum
                
        y_pred = x_sum_w.argmax(axis = 1) 
        return y_pred

# def regra_soma_padrao(fac_p_w_x, fou_p_w_x, kar_p_w_x, Pw):
# x_sum_w = np.empty((10, 2000))

# for i in range(10):
#     for k in range(2000):
#         views_sum =  fou_p_w_x[i,k] + kar_p_w_x[i,k]
#         # views_sum =  fac_p_w_x[i,k] fou_p_w_x[i,k] + kar_p_w_x[i,k]
#         x_sum_w[i,k] = (1-2)*Pw[i] + views_sum

# y_pred = x_sum_w.argmax(axis = 0) 
# return y_pred

## Validação cruzada

- **Observação**: No modelo gaussiano,  as desidades de probabilidades da VIEW1 (mfeat-fac) são todas zeradas, o que causa uma baixa na performance geral do modelo da regra da soma. Por esta razão, apenas neste modelo, ela foi desconsiderada. Portanto, no modelo gaussiano apresentamos os resultados os dados considerando somente VIEW2 e VIEW3.

- As médias das métricas para cada split da validação estão no arquivo **data/gaussian_training_data.csv**. Nota-se que muitos resultados aprensetam valores altos ou representações do infinito ("inf"). Isso se deu por conta de problemas de overflow na multiplicação usando valores muito pequenos fato que, aparentemente, foi causado pela quantidade de splits.

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


RANDOM_SEED = 42
FOLDS = 10
REPEATS = 30
REAPEATS_GAUSSIAN = 10

def get_splited_partition(idxs, y_true):
    partition = [[] for i in range(10)]
    
    for i,indice in enumerate(idxs):
        partition[y_true[indice]].append(i)
        
    return partition
    
cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)

train_precision_scores = np.empty((cv.get_n_splits(),))
train_recall_scores = np.empty((cv.get_n_splits(),))
train_f1_scores = np.empty((cv.get_n_splits(),))
train_acc_scores = np.empty((cv.get_n_splits(),))

test_precision_scores = np.empty((cv.get_n_splits(),))
test_recall_scores = np.empty((cv.get_n_splits(),))
test_f1_scores = np.empty((cv.get_n_splits(),))
test_acc_scores = np.empty((cv.get_n_splits(),))

#views = [fac, fou, kar]
views = [fou, kar]

for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
    print(f"{i+1}/{cv.get_n_splits()}", end=" ")
    
    test_views = [v[test_index] for v in views]
    train_views = [v[train_index] for v in views]
    
    local_partition = get_splited_partition(train_index, y_true)
    clf = RegraSomaClasificadorBayesiano(local_partition, Pw)
    
    clf.fit(train_views, y_true[train_index])
    
    y_pred_train = clf.predict(train_views)
    y_pred_test = clf.predict(test_views)
    
    scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")

    train_precision_scores[i] = scores_train[0]
    train_recall_scores[i] = scores_train[1]
    train_f1_scores[i] = scores_train[2]
    train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)

    scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                   y_pred_test, 
                                                   average="macro")

    test_precision_scores[i] = scores_test[0]
    test_recall_scores[i] = scores_test[1]
    test_f1_scores[i] = scores_test[2]
    test_acc_scores[i] = accuracy_score(y_true[test_index], y_pred_test)

    report = dict(
        runs = cv.get_n_splits(),
        split=i+1,
        train_accuracy = train_acc_scores[i],
        train_precision = train_precision_scores[i],
        train_recall = train_recall_scores[i],
        train_fscore = train_f1_scores.mean(),
        test_accuracy = test_acc_scores.mean(),
        test_precision = test_precision_scores[i],
        test_recall = test_recall_scores[i],
        test_fscore = test_f1_scores[i],

    )

    if os.path.exists("data/gaussian_300_splits.csv"):
        pd.DataFrame([report]).to_csv("data/gaussian_300_splits.csv", mode="a", header=False, decimal=",", index=False)
    else:
        pd.DataFrame([report]).to_csv("data/gaussian_300_splits.csv", mode="a", header=True, decimal=",", index=False)



/home/thiago/.anaconda3/envs/fuzzy/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass n_splits=10, n_repeats=30, random_state=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


1/300 2/300 3/300 4/300 5/300 6/300 7/300 8/300 9/300 10/300 11/300 12/300 13/300 14/300 15/300 16/300 17/300 18/300 19/300 20/300 21/300 22/300 23/300 24/300 25/300 26/300 27/300 28/300 29/300 30/300 31/300 32/300 33/300 34/300 35/300 36/300 37/300 38/300 39/300 

## Treinamento com base completa

- O objetivo aqui é determinar o desempenho do classificador por classe, tendo em vista diversas métricas de classificação. Observar os resultados no arquivo **data/gaussian_classification_report.txt**

In [ ]:
from sklearn.metrics import classification_report

#views = [fac, fou, kar]
views = [fou, kar]

clf = RegraSomaClasificadorBayesiano(partition, Pw)

clf.fit(views, y_true)

y_pred = clf.predict(views)

report = classification_report(y_true, y_pred, output_dict=False, digits=4)

acc_score_gaussian = accuracy_score(y_true, y_pred)

with open("data/gaussian_classification_report.txt", "w") as report_file:
    report_file.write(report)


## Estimativa pontual e intervalo de confiança

In [ ]:
p = acc_score_gaussian
z_padrao = 1.96 # Confiança de 95%

def calc_intervalo_confiança(p, z=z_padrao, n=2000):
    diff = z*np.sqrt(p*(1-p)/n)
    return (round(p - diff, 4), round(p+diff, 4))

print("Estimativa pontual: ", p)
print("Intervalo de confiança: ", calc_intervalo_confiança(p))

# ii) K-Vizinhos

## Normalizando 


In [ ]:
from sklearn.preprocessing import minmax_scale

fac_norm = minmax_scale(fac)
fou_norm = minmax_scale(fou)
kar_norm = minmax_scale(kar)

## Distâncias entre os elementos

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

fac_dist = euclidean_distances(fac_norm, fac_norm)
fou_dist = euclidean_distances(fou_norm, fou_norm)
kar_dist = euclidean_distances(kar_norm, kar_norm)

## Função de densidade

In [ ]:
def calc_knn_density_prob(view_dists, k, Pw, y_true):
    qtd_x = view_dists.shape[0]
    qtd_w = len(Pw)
    
    p_x_w = np.empty((qtd_x, qtd_w))
    k_vizinhos = view_dists.argsort(axis=1)[:,:k+1]
    
    for j in range(qtd_x):
        w_vizinhos = y_true[k_vizinhos[j, 1:]]
        for i in range(qtd_w):
            p_x_w[j,i] = (w_vizinhos == i).sum()/k
                    
    return p_x_w
    

## Desidades por dataset

In [ ]:
# fac_knn_density_probs = calc_knn_density_prob(fac_dist, 5, Pw, y_true)
# fou_knn_density_probs = calc_knn_density_prob(fou_dist, 5, Pw, y_true)
# kar_knn_density_probs = calc_knn_density_prob(kar_dist, 5, Pw, y_true)


## Prob. à posteriori por view

In [ ]:
# fac_knn_posteriori_probs = calc_prob_posteriori(fac_knn_density_probs, Pw)
# fou_knn_posteriori_probs = calc_prob_posteriori(fou_knn_density_probs, Pw)
# kar_knn_posteriori_probs = calc_prob_posteriori(kar_knn_density_probs, Pw)

## Regra da soma

In [ ]:
# y_pred_knn_all = regra_soma_padrao([fac_knn_posteriori_probs, 
#                                    fou_knn_posteriori_probs, 
#                                    kar_knn_posteriori_probs], Pw)

# print("Acurácia: ", accuracy_score(y_true, y_pred_knn_all))
# print("Medida-F: ", f1_score(y_true, y_pred_knn_all, average="macro"))

## Validação cruzada

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import os

cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)


k_range = range(3,4)
views = [fac, fou, kar]

for k in k_range:
    
    train_precision_scores = np.empty((cv.get_n_splits(),))
    train_recall_scores = np.empty((cv.get_n_splits(),))
    train_f1_scores = np.empty((cv.get_n_splits(),))

    test_precision_scores = np.empty((cv.get_n_splits(),))
    test_recall_scores = np.empty((cv.get_n_splits(),))
    test_f1_scores = np.empty((cv.get_n_splits(),))
    
    for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
        print(f"[k={k}] {i+1}/{cv.get_n_splits()}", end=" ")

        train_views = [minmax_scale(v[train_index]) for v in views]
        test_views = [minmax_scale(v[test_index]) for v in views]

        train_views_dists = [euclidean_distances(v,v) for v in train_views]
        test_views_dists = [euclidean_distances(v,v) for v in test_views]

    #     y_true_split = get_splited_partition(train_index, y_true)

        train_views_density_probs = [calc_knn_density_prob(v, k, Pw, y_true[train_index]) 
                                     for v in train_views_dists]

        train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in train_views_density_probs]

        y_pred_train = regra_soma_padrao(train_views_post_probs, Pw)


        test_views_density_probs = [calc_knn_density_prob(v, k, Pw, y_true[test_index]) 
                                     for v in test_views_dists]

        test_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in test_views_density_probs]

        y_pred_test = regra_soma_padrao(test_views_post_probs, Pw)
        
        scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")
        
#         train_precision_scores[i] = scores_train[0]
#         train_recall_scores[i] = scores_train[1]
#         train_f1_scores[i] = scores_train[2]

#         scores_test = precision_recall_fscore_support(y_true[test_index], 
#                                                        y_pred_test, 
#                                                        average="macro")
        
#         test_precision_scores[i] = scores_test[0]
#         test_recall_scores[i] = scores_test[1]
#         test_f1_scores[i] = scores_test[2]
        
        
        train_precision_scores[i] = scores_train[0]
        train_recall_scores[i] = scores_train[1]
        train_f1_scores[i] = scores_train[2]
        train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)

        scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                       y_pred_test, 
                                                       average="macro")

        test_precision_scores[i] = scores_test[0]
        test_recall_scores[i] = scores_test[1]
        test_f1_scores[i] = scores_test[2]
        test_acc_scores[i] = accuracy_score(y_true[test_index], y_pred_test)
        
        report = dict(
            runs = cv.get_n_splits(),
            split=i+1,
            train_accuracy = train_acc_scores[i],
            train_precision = train_precision_scores[i],
            train_recall = train_recall_scores[i],
            train_fscore = train_f1_scores[i],
            test_accuracy = test_acc_scores[i],
            test_precision = test_precision_scores[i],
            test_recall = test_recall_scores[i],
            test_fscore = test_f1_scores[i],
        )

        if os.path.exists("data/knn_300_splits.csv"):
            pd.DataFrame([report]).to_csv("data/knn_300_splits.csv", mode="a", header=False, decimal=",", index=False)
        else:
            pd.DataFrame([report]).to_csv("data/knn_300_splits.csv", mode="a", header=True, decimal=",", index=False)
        
#     report = dict(
#         k = k,
#         runs = cv.get_n_splits(),
#         train_precision = train_precision_scores.mean(),
#         train_precision_std = train_precision_scores.std(),
#         train_recall = train_recall_scores.mean(),
#         train_recall_std = train_recall_scores.std(),
#         train_fscore = train_f1_scores.mean(),
#         train_fscore_std = train_f1_scores.std(),
#         test_precision = test_precision_scores.mean(),
#         test_precision_std = test_precision_scores.std(),
#         test_recall = test_recall_scores.mean(),
#         test_recall_std = test_recall_scores.std(),
#         test_fscore = test_f1_scores.mean(),
#         test_fscore_std = test_f1_scores.std(),
        
#     )
    
#     if os.path.exists("data/knn_300_splits.csv"):
#         pd.DataFrame([report]).to_csv("data/knn_300_splits.csv", mode="a", header=False, decimal=",", index=False)
#     else:
#         pd.DataFrame([report]).to_csv("data/knn_300_splits.csv", mode="a", header=True, decimal=",", index=False)
    
        
        
        
#         if test_f1_scores[:i+1].mean() > best_data["f1_scores"].mean():
#             best_data["f1_scores"] = test_f1_scores[:i+1]
#             best_data["k"] = k
            
#             print(f"\nMelhor k: {k}")
#             print(f"Acurácia de treino parcial: {train_acc_scores[:i+1].mean():4} +/- ({train_acc_scores[:i+1].std():4})")
#             print(f"Medida-F de treino parcial: {train_f1_scores[:i+1].mean():4} +/- ({train_f1_scores[:i+1].std():4})\n")

#             print(f"Acurácia de test parcial: {test_acc_scores[:i+1].mean():4} +/- ({test_acc_scores[:i+1].std():4})")
#             print(f"Medida-F de test parcial: {test_f1_scores[:i+1].mean():4} +/- ({test_f1_scores[:i+1].std():4})\n")

    
    

## Treinamento com melhor valor de K

- O objetivo aqui é determinar o desempenho do classificador o valor de K com o qual ovtivemos a melhor medida-f média através validação cruzada. Observar os resultados no arquivo **data/knn_classification_report.txt**
- O melhor valor de K pode ser observado no arquivo **data/knn_best_k.csv**, gerado no experimento anterior

In [ ]:
from sklearn.metrics import classification_report

BEST_K = 3
#views = [fac, fou, kar]
views = [fac, fou, kar]

train_views = [minmax_scale(v) for v in views]

train_views_dists = [euclidean_distances(v,v) for v in train_views]

train_views_density_probs = [calc_knn_density_prob(v, BEST_K, Pw, y_true) 
                             for v in train_views_dists]

train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                          for v in train_views_density_probs]

y_pred = regra_soma_padrao(train_views_post_probs, Pw)
      
acc_score_knn = accuracy_score(y_true, y_pred)
    
report = classification_report(y_true, y_pred, output_dict=False, digits=4)

with open("data/knn_classification_report.txt", "w") as report_file:
    report_file.write(report)


## Estimativa pontual e intervalo de confiança

In [ ]:
p = acc_score_knn
z_padrao = 1.96 # Confiança de 95%

print("Estimativa pontual: ", p)
print("Intervalo de confiança: ", calc_intervalo_confiança(p, z_padrao))

# iii) Janela de Parzen

## Função de densidade

In [ ]:
def parzen_density_function(view, h, partition):
    qtd_x = view.shape[0]
    qtd_w = len(partition)
    
    p_x_w = np.empty((qtd_x, qtd_w))
    dims = view.shape[1]
    
    for i in range(qtd_w):
        n = len(partition[i])
        x_view = view[partition[i],:]
        for k in range(qtd_x):
            diff = (view[k] - x_view)/h
            gaussian_kernel = np.exp(-(diff**2)/2)/np.sqrt(2*np.pi)
            prod_dims = gaussian_kernel.prod(axis=1)
            p_x_w[k,i] = prod_dims.sum()/(n*h**dims)
            
    return p_x_w

In [ ]:
# fac_parzen_density_probs = parzen_density_function(fac, 2, partition)
# fou_parzen_density_probs = parzen_density_function(fou, 2, partition)
# kar_parzen_density_probs = parzen_density_function(kar, 2, partition)

## Prob. à posteriori por view

In [ ]:
# fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
# fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
# kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

## Regra da soma

In [ ]:
# for h in range(2, 21, 1):
#     fac_parzen_density_probs = parzen_density_function(fac, h, partition)
#     fou_parzen_density_probs = parzen_density_function(fou, h, partition)
#     kar_parzen_density_probs = parzen_density_function(kar, h, partition)

#     fac_prazen_posteriori_probs = calc_prob_posteriori(fac_parzen_density_probs, Pw)
#     fou_prazen_posteriori_probs = calc_prob_posteriori(fou_parzen_density_probs, Pw)
#     kar_prazen_posteriori_probs = calc_prob_posteriori(kar_parzen_density_probs, Pw)

#     y_pred_prazen_all = regra_soma_padrao([fac_prazen_posteriori_probs, 
#                                        fou_prazen_posteriori_probs, 
#                                        kar_prazen_posteriori_probs], Pw)

#     print(f"h: {h}")
#     print("Acurácia: ", accuracy_score(y_true, y_pred_prazen_all))
#     print("Medida-F: ", f1_score(y_true, y_pred_prazen_all, average="macro"))

## Validação cruzada

- Variamos h de 2 a 12 para fins de comparação. Não foi possíve, na validação cruzada, avaliar valores de janela maiores por limitação de tempo.

In [ ]:
cv = RepeatedStratifiedKFold(FOLDS, REPEATS, RANDOM_SEED)
train_acc_scores = np.empty((cv.get_n_splits(),))
train_f1_scores = np.empty((cv.get_n_splits(),))

test_acc_scores = np.empty((cv.get_n_splits(),))
test_f1_scores = np.empty((cv.get_n_splits(),))


h_range = range(2,3)
best_data = {"f1_scores": np.zeros((1,)), 
             "h":None}

views = [fac, fou, kar]

y_true = y_true.astype(int)
for h in h_range:
    train_precision_scores = np.empty((cv.get_n_splits(),))
    train_recall_scores = np.empty((cv.get_n_splits(),))
    train_f1_scores = np.empty((cv.get_n_splits(),))
    train_acc_scores = np.empty((cv.get_n_splits(),))

    test_precision_scores = np.empty((cv.get_n_splits(),))
    test_recall_scores = np.empty((cv.get_n_splits(),))
    test_f1_scores = np.empty((cv.get_n_splits(),))
    test_acc_scores = np.empty((cv.get_n_splits(),))
    
    for i, (train_index, test_index) in enumerate(cv.split(fac, y_true)):
        print(f"[h={h}] {i+1}/{cv.get_n_splits()}", end=" ")

        train_views = [v[train_index] for v in views]
        test_views = [v[test_index] for v in views]

        
        y_true_split = get_splited_partition(train_index, y_true)
        

        train_views_density_probs = [parzen_density_function(v, h, y_true_split) 
                                     for v in train_views]

        train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in train_views_density_probs]

        y_pred_train = regra_soma_padrao(train_views_post_probs, Pw)


        y_true_split = get_splited_partition(test_index, y_true)
        
        test_views_density_probs = [parzen_density_function(v, h, y_true_split) 
                                     for v in test_views]

        test_views_post_probs = [calc_prob_posteriori(v, Pw) 
                                  for v in test_views_density_probs]

        y_pred_test = regra_soma_padrao(test_views_post_probs, Pw)

        scores_train = precision_recall_fscore_support(y_true[train_index], 
                                                       y_pred_train, 
                                                       average="macro")

        train_precision_scores[i] = scores_train[0]
        train_recall_scores[i] = scores_train[1]
        train_f1_scores[i] = scores_train[2]
        train_acc_scores[i] = accuracy_score(y_true[train_index], y_pred_train)

        scores_test = precision_recall_fscore_support(y_true[test_index], 
                                                       y_pred_test, 
                                                       average="macro")

        test_precision_scores[i] = scores_test[0]
        test_recall_scores[i] = scores_test[1]
        test_f1_scores[i] = scores_test[2]
        test_acc_scores[i] = accuracy_score(y_true[test_index], y_pred_test)
        
        report = dict(
            runs = cv.get_n_splits(),
            split=i+1,
            train_accuracy = train_acc_scores[i],
            train_precision = train_precision_scores[i],
            train_recall = train_recall_scores[i],
            train_fscore = train_f1_scores[i],
            test_accuracy = test_acc_scores[i],
            test_precision = test_precision_scores[i],
            test_recall = test_recall_scores[i],
            test_fscore = test_f1_scores[i],
        )

        if os.path.exists("data/parzen_300_splits.csv"):
            pd.DataFrame([report]).to_csv("data/parzen_300_splits.csv", mode="a", header=False, decimal=",", index=False)
        else:
            pd.DataFrame([report]).to_csv("data/parzen_300_splits.csv", mode="a", header=True, decimal=",", index=False)

        
#         report = dict(
#             h = h,
#             runs = cv.get_n_splits(),
#             train_precision = train_precision_scores.mean(),
#             train_precision_std = train_precision_scores.std(),
#             train_recall = train_recall_scores.mean(),
#             train_recall_std = train_recall_scores.std(),
#             train_fscore = train_f1_scores.mean(),
#             train_fscore_std = train_f1_scores.std(),
#             test_precision = test_precision_scores.mean(),
#             test_precision_std = test_precision_scores.std(),
#             test_recall = test_recall_scores.mean(),
#             test_recall_std = test_recall_scores.std(),
#             test_fscore = test_f1_scores.mean(),
#             test_fscore_std = test_f1_scores.std(),
#         )

#     if os.path.exists("data/parzen_300_splits.csv"):
#         pd.DataFrame([report]).to_csv("data/parzen_300_splits.csv", mode="a", header=False, decimal=",", index=False)
#     else:
#         pd.DataFrame([report]).to_csv("data/parzen_300_splits.csv", mode="a", header=True, decimal=",", index=False)
    
    

## Treinamento com melhor valor de h

- O objetivo aqui é determinar o desempenho do classificador o valor de h com o qual ovtivemos a melhor medida-f média através validação cruzada. Observar os resultados no arquivo **data/parzen_classification_report_h2.txt**
- Contudo, assim como com a validação cruzada, o modelo obteve 100% em todas as medidas, para todos os valores de h e em ambas as bases de treinamento e validação.

- Valores de h a partir de 30 causaram underflow no trenamento graças à poderação exponecial feita no cálculo da função de densidade do modelo e portanto não puderam ser computadas.

In [ ]:
from sklearn.metrics import classification_report

BEST_H = 2
#views = [fac, fou, kar]
views = [fac, fou, kar]

train_views_density_probs = [parzen_density_function(v, BEST_H, partition) 
                             for v in views]

train_views_post_probs = [calc_prob_posteriori(v, Pw) 
                          for v in train_views_density_probs]

y_pred = regra_soma_padrao(train_views_post_probs, Pw)

report = classification_report(y_true, y_pred, output_dict=False, digits=4)

acc_score_parzen = accuracy_score(y_true, y_pred)

with open(f"data/parzen_classification_report_h{BEST_H}.txt", "w") as report_file:
    report_file.write(report)


## Estimativa pontual e intervalo de confiança

In [ ]:
p = acc_score_parzen
z_padrao = 1.96 # Confiança de 95%

print("Estimativa pontual: ", p)
print("Intervalo de confiança: ", calc_intervalo_confiança(p, z_padrao))